In [1]:
import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

import numpy as np
import math

from sympy.logic import POSform, SOPform
from sympy import symbols

symA = 'a0 a1 a2 a3 a4 a5 a6 a7 a8 a9 a10 a11 a12 a13 a14 a15'
symB = 'b0 b1 b2 b3 b4 b5 b6 b7 b8 b9 b10 b11 b12 b13 b14 b15'

a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15 = symbols(symA)
b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15 = symbols(symB)

A = [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15]

B = [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15]

def toBin (val, size):
    return list(reversed(list(map(lambda x: int(x),'{:0{}b}'.format(val, size)))))


In [2]:
def build_X_y(NBITS, idxOfBitToEncode, hypothesis):
    X = []
    y = []
    for v1 in range(2**NBITS):
        binv1 = toBin(v1, NBITS)
        include = True
        for hyp in hypothesis:
            if binv1[hyp[0]] != hyp[1]:
                include = False
                break
        if not include:
            continue
        mathval1 = round(math.sqrt(v1))
        binval1 = toBin(mathval1, NBITS)
        targetbit = binval1[idxOfBitToEncode]
        print(v1, binv1, mathval1, binval1, targetbit)
        X.append(binv1)
        y.append(targetbit)
    print(X)
    print(y)
    return X, y

In [14]:
NBITS = 4
idxOfBitToEncode = 2
hypothesis = []  # Example hypothesis to filter inputs

X, y = build_X_y(NBITS, idxOfBitToEncode, hypothesis)

0 [0, 0, 0, 0] 0 [0, 0, 0, 0] 0
1 [1, 0, 0, 0] 1 [1, 0, 0, 0] 0
2 [0, 1, 0, 0] 1 [1, 0, 0, 0] 0
3 [1, 1, 0, 0] 2 [0, 1, 0, 0] 0
4 [0, 0, 1, 0] 2 [0, 1, 0, 0] 0
5 [1, 0, 1, 0] 2 [0, 1, 0, 0] 0
6 [0, 1, 1, 0] 2 [0, 1, 0, 0] 0
7 [1, 1, 1, 0] 3 [1, 1, 0, 0] 0
8 [0, 0, 0, 1] 3 [1, 1, 0, 0] 0
9 [1, 0, 0, 1] 3 [1, 1, 0, 0] 0
10 [0, 1, 0, 1] 3 [1, 1, 0, 0] 0
11 [1, 1, 0, 1] 3 [1, 1, 0, 0] 0
12 [0, 0, 1, 1] 3 [1, 1, 0, 0] 0
13 [1, 0, 1, 1] 4 [0, 0, 1, 0] 1
14 [0, 1, 1, 1] 4 [0, 0, 1, 0] 1
15 [1, 1, 1, 1] 4 [0, 0, 1, 0] 1
[[0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 0], [1, 0, 1, 0], [0, 1, 1, 0], [1, 1, 1, 0], [0, 0, 0, 1], [1, 0, 0, 1], [0, 1, 0, 1], [1, 1, 0, 1], [0, 0, 1, 1], [1, 0, 1, 1], [0, 1, 1, 1], [1, 1, 1, 1]]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [ ]:
tree = DecisionTreeClassifier().fit(X, y) # max_depth=2


[[[13.  3.]]

 [[ 8.  0.]]

 [[ 5.  3.]]

 [[ 4.  0.]]

 [[ 1.  3.]]

 [[ 1.  1.]]

 [[ 1.  0.]]

 [[ 0.  1.]]

 [[ 0.  2.]]]


In [16]:
r = export_text(tree) # , feature_names = ["a", "b"])

print (r)


|--- feature_3 <= 0.50
|   |--- class: 0
|--- feature_3 >  0.50
|   |--- feature_2 <= 0.50
|   |   |--- class: 0
|   |--- feature_2 >  0.50
|   |   |--- feature_1 <= 0.50
|   |   |   |--- feature_0 <= 0.50
|   |   |   |   |--- class: 0
|   |   |   |--- feature_0 >  0.50
|   |   |   |   |--- class: 1
|   |   |--- feature_1 >  0.50
|   |   |   |--- class: 1



In [17]:
def build_abstract_tree(tree, node_id=0):
    """
    Recursively builds an abstract binary tree from a DecisionTreeClassifier.
    Each node is a dict with 'feature', 'threshold', 'left', 'right', or 'value' for leaves.
    """
    if tree.children_left[node_id] == tree.children_right[node_id]:  # Leaf node
        return {'value': tree.value[node_id][0].tolist()}
    return {
        'feature': tree.feature[node_id],
        'threshold': tree.threshold[node_id],
        'value': tree.value[node_id][0].tolist(),
        'left': build_abstract_tree(tree, tree.children_left[node_id]),
        'right': build_abstract_tree(tree, tree.children_right[node_id])
    }

# Usage:
abstract_tree = build_abstract_tree(tree.tree_)
print(abstract_tree)

{'feature': 3, 'threshold': 0.5, 'value': [13.0, 3.0], 'left': {'value': [8.0, 0.0]}, 'right': {'feature': 2, 'threshold': 0.5, 'value': [5.0, 3.0], 'left': {'value': [4.0, 0.0]}, 'right': {'feature': 1, 'threshold': 0.5, 'value': [1.0, 3.0], 'left': {'feature': 0, 'threshold': 0.5, 'value': [1.0, 1.0], 'left': {'value': [1.0, 0.0]}, 'right': {'value': [0.0, 1.0]}}, 'right': {'value': [0.0, 2.0]}}}}


In [19]:
def print_abstract_tree(node, depth=0):
    indent = "  " * depth
    if 'value' in node and ('feature' not in node):
        print(f"{indent}Leaf: {node['value']}")
    else:
        print(f"{indent}Feature {node['feature']} <= {node['threshold']}, value={node['value']}")
        print_abstract_tree(node['left'], depth + 1)
        print_abstract_tree(node['right'], depth + 1)

# Affichage
print_abstract_tree(abstract_tree)

Feature 3 <= 0.5, value=[13.0, 3.0]
  Leaf: [8.0, 0.0]
  Feature 2 <= 0.5, value=[5.0, 3.0]
    Leaf: [4.0, 0.0]
    Feature 1 <= 0.5, value=[1.0, 3.0]
      Feature 0 <= 0.5, value=[1.0, 1.0]
        Leaf: [1.0, 0.0]
        Leaf: [0.0, 1.0]
      Leaf: [0.0, 2.0]


In [18]:
print (dir(tree.tree_))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'apply', 'capacity', 'children_left', 'children_right', 'compute_feature_importances', 'compute_node_depths', 'compute_partial_dependence', 'decision_path', 'feature', 'impurity', 'max_depth', 'max_n_classes', 'missing_go_to_left', 'n_classes', 'n_features', 'n_leaves', 'n_node_samples', 'n_outputs', 'node_count', 'predict', 'threshold', 'value', 'weighted_n_node_samples']


In [20]:
def generate_c_code(node, depth=0):
    indent = "    " * depth
    code = ""
    if 'value' in node and ('feature' not in node):
        # On suppose une classification binaire, retourne la classe majoritaire
        val = node['value']
        result = 0 if val[0] >= val[1] else 1
        code += f"{indent}return {result};\n"
    else:
        code += f"{indent}if (features[{node['feature']}] <= {node['threshold']}) {{\n"
        code += generate_c_code(node['left'], depth + 1)
        code += f"{indent}}} else {{\n"
        code += generate_c_code(node['right'], depth + 1)
        code += f"{indent}}}\n"
    return code

def tree_to_c_function(abstract_tree, func_name="predict"):
    c_code = f"int {func_name}(float features[]) {{\n"
    c_code += generate_c_code(abstract_tree)
    c_code += "}\n"
    return c_code

# Exemple d'utilisation
c_function_code = tree_to_c_function(abstract_tree)
print(c_function_code)

int predict(float features[]) {
if (features[3] <= 0.5) {
    return 0;
} else {
    if (features[2] <= 0.5) {
        return 0;
    } else {
        if (features[1] <= 0.5) {
            if (features[0] <= 0.5) {
                return 0;
            } else {
                return 1;
            }
        } else {
            return 1;
        }
    }
}
}

